In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
import os
from tqdm.notebook import tqdm
from data_processing.GameFeaturesGenerator import GameFeaturesGenerator
from data_processing.TournamentDataProcessor import TournamentDataProcessor
from data_processing.LeaguesDataProcessor import LeaguesDataProcessor
from data_processing.utils.download_functions import *
os.chdir('esports-data')
os.listdir()

['2021_raw_game_data',
 'games',
 'leagues.json',
 'mapping_data.json',
 'players.json',
 'teams.json',
 'tournaments.json']

In [2]:
# Read in teams data
with open("teams.json", "r") as json_file:
   teams_data = json.load(json_file)

team_df = []
for team in teams_data:
    team_df.append({'team_id': team['team_id'], 'team_name': team['name']})
    
team_df = pd.DataFrame(team_df)

In [3]:
# Read in tournament data
with open("tournaments.json", "r") as json_file:
   tournament_data_all = json.load(json_file)

# Read in mappings data
with open("mapping_data.json", "r") as json_file:
   mappings_data = json.load(json_file)
   
mappings = {
   esports_game["esportsGameId"]: esports_game for esports_game in mappings_data
}

# Set up LeaguesDataProcessor
leagues_data_processor = LeaguesDataProcessor()

In [8]:
tournaments_2021 = [x['slug'] for x in tournament_data_all if '2022' in x['slug']]
# Get rid of international tournaments because they need to be processed separately 
# tournaments_2021.remove('msi_2021')
# tournaments_2021.remove('worlds_2021')

# tournaments_2021 = ['nlc_2021_split1', 'lla_closing_2021', 'baltic_masters_2021_summer', 'lcs_academy_summer_2021', 'belgian_league_2021_split1', 'dutch_league_2021_split2', 'greek_legends_league_summer_2021', 'lck_regional_finals_2021', 'ljl_academy_2021', 'lcs_lock_in_2021', 'vcs_winter_2021', 'lcs_proving_grounds_summer_2021', 'ultraliga_summer_2021', 'cblol_split_2_2021', 'tcl_summer_2021', 'liga_portuguesa_spring_2021', 'cblol_split_1_2021', 'hitpoint_masters_spring_2021', 'lck_spring_2021', 'lec_summer_2021', 'dutch_league_2021_split1', 'ljl_summer_2021', 'superliga_spring_2021', 'ljl_spring_2021', 'esports_balkan_league_spring_2021', 'lfl_2021_spring', 'european_masters_summer_2021_play_ins', 'tcl_winter_2021', 'european_masters_summer_2021_main_event', 'lcl_spring_2021', 'pg_nationals_spring_2021', 'lla_promotion_2021', 'european_masters_spring_2021_play_ins', 'cblol_academy_split_1_2021', 'lla_opening_2021', 'mss_2021', 'greek_legends_league_spring_2021', 'baltic_masters_2021_spring']
 
tournaments_2021 = ['ljl_academy_2022', 'elite_series_spring_2022', 'lco_split_1_2022', 'tcl_summer_2022', 'european_masters_spring_2022_main_event', 'nlc_spring_2022', 'stars_league_closing_2022', 'honor_league_opening_2022', 'honor_division_opening_2022', 'honor_league_closing_2022', 'lfl_2022_spring', 'lcs_academy_spring_2022', 'lck_spring_2022', 'lfl_2022_summer', 'volcano_league_closing_2022', 'tal_winter_2022', 'elite_series_summer_2022', 'cblol_split_2_2022', 'lla_opening_2022', 'prime_league_spring_2022', 'liga_portuguesa_spring_2022', 'european_masters_summer_2022_main_event', 'golden_league_closing_2022', 'master_flow_league_opening_2022', 'lcs_spring_2022', 'lla_closing_2022', 'master_flow_league_closing_2022', 'nlc_summer_2022', 'esports_balkan_league_summer_2022', 'european_masters_spring_2022_play_ins', 'hitpoint_masters_spring_2022', 'ljl_spring_2022', 'golden_league_opening_2022', 'pg_nationals_summer_2022', 'lcs_amateur_circuit_summer_2022', 'college_championship_2022']
 
tournaments = [x for x in tournament_data_all if x['slug'] in tournaments_2021]
tournaments_names = [x['slug'] for x in tournaments]
print(tournaments_names)


['lcs_amateur_circuit_summer_2022', 'ljl_academy_2022', 'elite_series_spring_2022', 'lco_split_1_2022', 'tcl_summer_2022', 'european_masters_spring_2022_main_event', 'nlc_spring_2022', 'stars_league_closing_2022', 'honor_league_opening_2022', 'honor_division_opening_2022', 'honor_league_closing_2022', 'lfl_2022_spring', 'lcs_academy_spring_2022', 'lck_spring_2022', 'lfl_2022_summer', 'volcano_league_closing_2022', 'tal_winter_2022', 'elite_series_summer_2022', 'cblol_split_2_2022', 'lla_opening_2022', 'prime_league_spring_2022', 'liga_portuguesa_spring_2022', 'european_masters_summer_2022_main_event', 'golden_league_closing_2022', 'master_flow_league_opening_2022', 'lcs_spring_2022', 'lla_closing_2022', 'master_flow_league_closing_2022', 'nlc_summer_2022', 'esports_balkan_league_summer_2022', 'european_masters_spring_2022_play_ins', 'hitpoint_masters_spring_2022', 'ljl_spring_2022', 'golden_league_opening_2022', 'pg_nationals_summer_2022', 'lcs_amateur_circuit_summer_2022', 'college_ch

In [4]:
# tournaments = [x for x in tournament_data_all if x['slug'] in ['lec_summer_2021']]

In [9]:
"""
Iterate through each tournament and process the data, we will get two dataframes 

tournament_rows = [match_id, esportsGameId, league, team_id_1, team_id_2, outcome_1, outcome_2]
game_rows = [platformGameId, esportsGameId, team_id, start_time, outcome, ...features..., ...league_indicators...]]

Save these dataframes to csv files in the data folder with the corresponding tournament name 
"""

for tournament_data in tournaments:
    tournament_name = tournament_data['slug']
    print("Processing: " + tournament_name)
    # Set up TournamentDataProcessor
    try:
        tournament_data_processor = TournamentDataProcessor(tournament_data, leagues_data_processor.leagues_df)
    except IndexError as e:
        print(f"Error processing tournament {tournament_name}. Tournament could not be found in leagues data")
        continue
    # tournament_data_processor.get_tournament_stages()
    training_data, _ = tournament_data_processor.get_tournament_data(training_stages=[], testing_stages=[])  # Consider all stages to be training 
    print("Games in tournament: " + str(len(training_data)))
    
    # Now download each game, process them, and add them into game_rows 
    directory = "games"
    if not os.path.exists(directory):
      os.makedirs(directory)
    
    # Load each game and process them
    game_rows = []
    for i in tqdm(range(len(training_data))):
        try:
            game_id = training_data.iloc[i]['esportsGameId']
            game_mapping_data = mappings[game_id]
            platform_game_id = game_mapping_data['platformGameId']
            download_gzip_and_write_to_json(f"{directory}/{platform_game_id}")
            with open(f"games/{platform_game_id}.json", "r") as json_file:
                game_data = json.load(json_file)
        except KeyError:
            print(f"Match {game_id} was not found")
            continue
        try:
            game_features = GameFeaturesGenerator(game_data, game_mapping_data).process_game()
        except Exception as e:
            print(f"Error processing game {game_id}")
            print(e)
            continue
        game_features['league'] = training_data.iloc[i]['league']
        game_rows.append(game_features)
    
    game_rows = pd.concat(game_rows)
    game_rows_leagues = leagues_data_processor.transform_league_col(game_rows['league'])
    game_rows = pd.concat([game_rows.drop(['league'],axis=1).reset_index(), game_rows_leagues], axis=1)
    
    # Save to csv in the data folder
    game_data_directory = "2022_raw_game_data"
    if not os.path.exists(game_data_directory):
      os.makedirs(game_data_directory)
    training_data.to_csv(f'{game_data_directory}/{tournament_name}_tournament_rows.csv', index=False)
    game_rows.to_csv(f'{game_data_directory}/{tournament_name}_game_rows.csv', index=False)
    
    # Delete the temp directory
    shutil.rmtree(directory)

Processing: lcs_amateur_circuit_summer_2022
Error processing tournament lcs_amateur_circuit_summer_2022. Tournament could not be found in leagues data
Processing: ljl_academy_2022
Games in tournament: 39


  0%|          | 0/39 [00:00<?, ?it/s]

File opened
games/ESPORTSTMNT05:2980324.json written
File opened
games/ESPORTSTMNT05:2941748.json written
File opened
games/ESPORTSTMNT05:2941719.json written
File opened
games/ESPORTSTMNT03:2883648.json written
File opened
games/ESPORTSTMNT05:2980363.json written
File opened
games/ESPORTSTMNT01:3093541.json written
File opened
games/ESPORTSTMNT03:2886243.json written
File opened
games/ESPORTSTMNT03:2886272.json written
File opened
games/ESPORTSTMNT03:2886224.json written
File opened
games/ESPORTSTMNT05:2980358.json written
File opened
games/ESPORTSTMNT03:2883589.json written
File opened
games/ESPORTSTMNT05:2941764.json written
File opened
games/ESPORTSTMNT01:3093524.json written
File opened
games/ESPORTSTMNT03:2883627.json written
File opened
games/ESPORTSTMNT01:3093490.json written
File opened
games/ESPORTSTMNT05:2941803.json written
File opened
games/ESPORTSTMNT05:2980355.json written
Match 108216929094512004 was not found
File opened
games/ESPORTSTMNT03:2886203.json written
File op

  0%|          | 0/73 [00:00<?, ?it/s]

File opened
games/ESPORTSTMNT01:2703952.json written
File opened
games/ESPORTSTMNT01:2706576.json written
File opened
games/ESPORTSTMNT05:2520621.json written
File opened
games/ESPORTSTMNT05:2530486.json written
File opened
games/ESPORTSTMNT01:2707442.json written
File opened
games/ESPORTSTMNT05:2520914.json written
File opened
games/ESPORTSTMNT01:2702962.json written
File opened
games/ESPORTSTMNT01:2707372.json written
File opened
games/ESPORTSTMNT05:2520911.json written
File opened
games/ESPORTSTMNT01:2703026.json written
File opened
games/ESPORTSTMNT05:2520617.json written
File opened
games/ESPORTSTMNT05:2520434.json written
File opened
games/ESPORTSTMNT01:2706603.json written
File opened
games/ESPORTSTMNT01:2702939.json written
File opened
games/ESPORTSTMNT05:2520638.json written
File opened
games/ESPORTSTMNT05:2520889.json written
File opened
games/ESPORTSTMNT01:2694052.json written
File opened
games/ESPORTSTMNT05:2530478.json written
File opened
games/ESPORTSTMNT01:2694167.json w

...

File opened
games/ESPORTSTMNT03:2547501.json written
File opened
games/ESPORTSTMNT03:2544713.json written
File opened
games/ESPORTSTMNT03:2568909.json written
File opened
games/ESPORTSTMNT02:2558507.json written
File opened
games/ESPORTSTMNT03:2546191.json written
File opened
games/ESPORTSTMNT03:2549959.json written
File opened
games/ESPORTSTMNT02:2578803.json written
File opened
games/ESPORTSTMNT02:2578825.json written
Match 107455808282935222 was not found
File opened
games/ESPORTSTMNT03:2546235.json written
File opened
games/ESPORTSTMNT03:2568860.json written
File opened
games/ESPORTSTMNT03:2564422.json written
File opened
games/ESPORTSTMNT02:2558521.json written
File opened
games/ESPORTSTMNT03:2546265.json written
File opened
games/ESPORTSTMNT03:2567089.json written
File opened
games/ESPORTSTMNT03:2580010.json written
File opened
games/ESPORTSTMNT02:2558499.json written
File opened
games/ESPORTSTMNT03:2568852.json written
File opened
games/ESPORTSTMNT03:2544631.json written
File op

  0%|          | 0/71 [00:00<?, ?it/s]

Match 108166853211545715 was not found
File opened
games/ESPORTSTMNT02:2785660.json written
File opened
games/ESPORTSTMNT02:2776167.json written
File opened
games/ESPORTSTMNT03:2793512.json written
File opened
games/ESPORTSTMNT03:2798791.json written
File opened
games/ESPORTSTMNT03:2794305.json written
File opened
games/ESPORTSTMNT02:2810705.json written
File opened
games/ESPORTSTMNT02:2786478.json written
File opened
games/ESPORTSTMNT02:2810780.json written
File opened
games/ESPORTSTMNT03:2794361.json written
File opened
games/ESPORTSTMNT02:2785811.json written
File opened
games/ESPORTSTMNT03:2798889.json written
File opened
games/ESPORTSTMNT03:2783554.json written
File opened
games/ESPORTSTMNT03:2798840.json written
File opened
games/ESPORTSTMNT02:2810757.json written
File opened
games/ESPORTSTMNT03:2794453.json written
File opened
games/ESPORTSTMNT03:2793542.json written
File opened
games/ESPORTSTMNT02:2810341.json written
File opened
games/ESPORTSTMNT02:2785788.json written
File op

  0%|          | 0/77 [00:00<?, ?it/s]

Match 107898708100265998 was not found
Match 107904811182472292 was not found
Match 107898708100266000 was not found
Match 107904811182472294 was not found
Match 107904811182472295 was not found
Match 107898708100266002 was not found
Match 107904811182472296 was not found
Match 107898708100266004 was not found
Match 107904811182472298 was not found
Match 107904811182472299 was not found
Match 107898708100266006 was not found
Match 107904811182472300 was not found
Match 107898708100266008 was not found
Match 107904811182472302 was not found
Match 107904811182472303 was not found
Match 107898708100266010 was not found
Match 107904811182472304 was not found
Match 107898708100266012 was not found
Match 107904811182472306 was not found
Match 107898708100266014 was not found
Match 107904811182472308 was not found
Match 107898708100266016 was not found
Match 107904811182472310 was not found
Match 107904811182472311 was not found
Match 107898708100266018 was not found
Match 107904811182472312 

In [14]:
tournament_data.keys()

dict_keys(['id', 'leagueId', 'name', 'slug', 'sport', 'startDate', 'endDate', 'stages'])

In [6]:
leagues_data_processor.leagues_df

,league_id,league_name,league_region,league_tournaments
0,98767991299243165,LCS,NORTH AMERICA,110303581083678395
1,98767991299243165,LCS,NORTH AMERICA,109517090066605615
2,98767991299243165,LCS,NORTH AMERICA,108206581962155974
3,98767991299243165,LCS,NORTH AMERICA,107458367237283414
4,98767991299243165,LCS,NORTH AMERICA,107458335260330212
...,...,...,...,...
5,105549980953490846,CBLOL Academy,BRAZIL,105562692792011783
0,110371976858004491,North Regional League,LATIN AMERICA NORTH,110372140231846916
0,110372322609949919,South Regional League,LATIN AMERICA SOUTH,110372454557117766
0,108001239847565215,TFT Rising Legends,INTERNATIONAL,109761195185432372


In [7]:
[x for x in leagues_data_processor.leagues_df['league_id'] if x=='101097443346691685']

[]

In [9]:
np.sort(leagues_data_processor.leagues_df['league_id'])

array(['100695891328981122', '100695891328981122', '100695891328981122',
       '100695891328981122', '100695891328981122', '100695891328981122',
       '100695891328981122', '100695891328981122', '100695891328981122',
       '100695891328981122', '100695891328981122', '100695891328981122',
       '100695891328981122', '100695891328981122', '101382741235120470',
       '101382741235120470', '101382741235120470', '101382741235120470',
       '101382741235120470', '101382741235120470', '101382741235120470',
       '101382741235120470', '101382741235120470', '101382741235120470',
       '101382741235120470', '101382741235120470', '104366947889790212',
       '104366947889790212', '104366947889790212', '104366947889790212',
       '104366947889790212', '104366947889790212', '104366947889790212',
       '104366947889790212', '104366947889790212', '104366947889790212',
       '105266074488398661', '105266074488398661', '105266074488398661',
       '105266074488398661', '105266074488398661', 